In [1]:
#%matplotlib inline
import sys
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/weather_mast/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/MEPS/')
sys.path.append('/Volumes/SANDISK128/Documents/Thesis/Python/Retrieval_MEPS/')
import netCDF4
import numpy as np
import matplotlib.pyplot as plt
import datetime
import math

import createFolder as cF
import calc_date as cd
import plot_sfc_spaghetti_ret as spagh
import save_fig as SF
import get_Haukeli_obs_data as obsDat
import calc_48h_acc as acc
import fill_values as fv
import plot_vertical as pvert

import os

import pandas as pd
import matplotlib as mpl
mpl.style.use('ggplot')

from calc_diff_obs_model import *
from plt_spaghetti_scatter import *

In [2]:
year = 2016
month = 12
tid = '00'
station = 'Haukeliseter'
savefig = 1 # 1 = yes, 0 = no

In [3]:
#### days 
t = ['20', '21', '22', '23', '24', '25', '26', '27']


In [4]:
#var = 'PP'
#var = 'SP'
#var = 'T2'
#var = 'WD'
var = 'WS'
if var == 'PP':
    var_name = 'precipitation_amount_acc'
elif var == 'SP':
    var_name = 'air_pressure_at_sea_level'
elif var == 'T2':
    var_name = 'air_temperature_2m'
elif var == 'WD':
    var_name = 'wind_direction'
elif var == 'WS':
    var_name = 'wind_speed'


figdir_var = '../../Figures/MEPS/%s' %(var_name)
figdir_scatter = '../../Figures/MEPS/scatter_MEPS_obs/%s' %(var_name)
figdir_error = '../../Figures/MEPS/error/%s' %(var_name)
if savefig == 1:
    cF.createFolder('%s/' %(figdir_var))
    cF.createFolder('%s/' %(figdir_scatter))
    cF.createFolder('%s/' %(figdir_error))




form = 'png'

In [5]:
### MEPS
if var == 'SP' or var == 'PP':
    level = 'sfc'
elif var == 'T2':
    level = '2m'
elif var == 'WD' or var == 'WS':
    level = '10m'

ncdir_var = '../../Data/MEPS/%s/%s_%s' %(station,level,tid)



################################################################
### Haukeli obs.
txtdir = '../../Data/eklima/'
txt_filename = 'Haukeli_obs_Dec2016'

In [6]:
# read in the Haukeliseter time file, sorted by daily values
Haukeli_Date = pd.read_csv('%s/Observations-Date-Hour.txt' %(txtdir),\
                    sep = ';',header=19, skipfooter = 11, engine= 'python')
DateHour = Haukeli_Date['Date-Hour(UTC)']

In [7]:
# read in the Haukeliseter observation file, sorted by daily values
Haukeli = pd.read_csv('%s/%s.txt' %(txtdir,txt_filename),\
                    sep = ';',header=18, skipfooter = 2, engine= 'python')

if var == 'PP':
    obsvar = Haukeli['RR_1'].astype(float)         # Precipitation (1 hour); [mm]
elif var == 'T2':
    obsvar = Haukeli['TA'].astype(float)           # Air temperature;degC --> Lufttemperatur ved observasjonstiden i 2 m høyde
elif var == 'SP':
    obsvar = Haukeli['PR'].astype(float)           # PR;Air pressure at sea level;hPa
elif var == 'WD' or var == 'WS':
    obsvar_DD = Haukeli['DD'].astype(float)             # Wind direction (FF); [degrees]
    obsvar_FF = Haukeli['FF'].astype(float)             # Wind speed (10 meters above ground);m/s

if var == 'PP' or var == 'T2' or var == 'SP':
    obsvar0, dt_obsvar0, hour_obsvar0, day_obsvar0, month_obsvar0, year_obsvar0 = obsDat.valid_values(obsvar, DateHour)
elif var == 'WD' or var == 'WS':
    idxFF = np.where(obsvar_FF[:] == -9999)
    idxDD = np.where(obsvar_DD[:] == -9999)

    obsvar_FF[idxFF[0][:]] = np.nan
    obsvar_DD[idxDD[0][:]] = np.nan

    ### calculate the U, V wind component for barb plot
    # http://colaweb.gmu.edu/dev/clim301/lectures/wind/wind-uv.html
    # first calculate the mathematical wind direction in deg
    md_deg = 270 - obsvar_DD
    for k in range(0,md_deg.shape[0]):
        if md_deg[k] <0 :
            md_deg[k] = md_deg[k] +360
    md_rad = math.pi/180. * md_deg
    uwind = obsvar_FF*np.cos(md_rad)
    vwind = obsvar_FF*np.sin(md_rad)
    
    obsvar_FF_wind, obsvar_DD_wind, \
            obsvar_dt_wind, obsvar_hour_wind, \
            obsvar_day_wind, obsvar_month_wind, \
            obsvar_year_wind = obsDat.valid_values_wind(obsvar_FF, obsvar_DD, DateHour)
    if var == 'WD':
        obsvar0 = obsvar_DD_wind
    elif var == 'WS':
        obsvar0 = obsvar_FF_wind

In [8]:
fn_var = dict()

if var == 'PP' or var == 'T2' or var == 'SP':
    model_var = dict()
elif var == 'WD' or var == 'WS':
    model_var_x = dict()
    model_var_y = dict()
    
    model_var = dict()


time_var      = dict()
lead_time_var = dict()
ini_day       = dict()
calday        = dict()
calmon        = dict()


In [9]:
calendar_day = []
initialisation_day = []
calendar_month = []
calendar_year = []
initialisation_hh = []

In [10]:
diff_var = dict()
Diff_var_0 = []
Diff_var_1 = []
Diff_var_2 = []
Diff_var_3 = []
Diff_var_4 = []
Diff_var_5 = []
Diff_var_6 = []
Diff_var_7 = []
Diff_var_8 = []
Diff_var_9 = []


all_day_max_var = []
all_day_min_var = []

In [11]:
model_var_0 = []
model_var_1 = []
model_var_2 = []
model_var_3 = []
model_var_4 = []
model_var_5 = []
model_var_6 = []
model_var_7 = []
model_var_8 = []
model_var_9 = []

obs_var = []

In [12]:
#t = ['24']
for day in t:
    
    if var == 'T2' or var == 'SP' or var == 'WD' or var == 'WS':
        ### connect variable data for two days
        df1 = obsvar0[int(day)-1]
        df2 = obsvar0[int(day)]
        Variable_obs = np.concatenate((df1,df2),axis=0)
        df3 = obsvar0[int(day)+1][0]
        Variable_obs = np.append(Variable_obs,df3)
    



    
    
    for ens_memb in range(0,10):
        fn_var[ens_memb] = netCDF4.Dataset('%s/%s%s%s_%s_%s.nc' %(ncdir_var,year,month,day,tid,ens_memb))
        
        time_var[ens_memb] = fn_var[ens_memb].variables['time']
        time_var[ens_memb] = fv.fill_nan(time_var[ens_memb][:])
        lead_time_var[ens_memb] = np.arange(0,time_var[ens_memb].shape[0])
# initialisation time for plotting
        ini_day = (datetime.datetime.utcfromtimestamp(time_var[0][0]).day)   # day of initialisation
        hh = (datetime.datetime.utcfromtimestamp(time_var[0][0]).hour)       # first hour of initialisation
        calday, calmon = cd.get_dayname(year, month, ini_day)        
# Read in the variable   
        if var == 'PP' or var == 'T2' or var == 'SP':
            model_var[ens_memb] = fn_var[ens_memb].variables[var_name][:]
            if var == 'T2':
                model_var[ens_memb] = model_var[ens_memb]-273.15
            elif var == 'SP':
                model_var[ens_memb] = model_var[ens_memb]/100
            #substitute missing values with nan 
            model_var[ens_memb] = fv.fill_nan(model_var[ens_memb][:])

          

        elif var == 'WD' or var == 'WS':
            model_var_x[ens_memb] = fn_var[ens_memb].variables['x_wind_10m'][:]
            model_var_y[ens_memb] = fn_var[ens_memb].variables['y_wind_10m'][:]
            #substitute missing values with nan 
            model_var_x[ens_memb] = fv.fill_nan(model_var_x[ens_memb][:,:])
            model_var_y[ens_memb] = fv.fill_nan(model_var_y[ens_memb][:,:])


            #### calculate wind direction / speed
            if var == 'WD':
                model_var[ens_memb] = np.arctan2(model_var_y[ens_memb][:,0], model_var_x[ens_memb][:,0])
                model_var[ens_memb] = 270-np.rad2deg(model_var[ens_memb])  # convert from math to meteo direction and to degrees
                larger360 = np.where(model_var[ens_memb] >360.)
                model_var[ens_memb][larger360] = model_var[ens_memb][larger360]-360
            elif var == 'WS':
                model_var[ens_memb] = np.sqrt((model_var_x[ens_memb][:,0])**2 + (model_var_y[ens_memb][:,0])**2) 

#### calculate every 60 min the accumulation over 48 hours
    if var == 'PP':
        Variable_obs = acc.accumulation_dt60_for48h(obsvar0, ini_day)
################

#### calculate the ensemble mean of variable every 3 hours (all 10 ensemble member averaged)
    val = np.zeros(shape=(10,np.asarray(model_var[0])[::3].shape[0]))
    model_var_std = []
    model_var_mean = []
    for ens_memb in range(0,10):
        if var == 'PP' or var == 'T2' or var == 'SP':
            val[ens_memb] = np.asarray(model_var[ens_memb])[::3,0]
        elif var == 'WD' or var == 'WS':
            val[ens_memb] = np.asarray(model_var[ens_memb])[::3]
          
    ### std, mean
    model_var_std.append(np.nanstd(val,axis=0,ddof=1))
    model_var_mean.append(np.mean(val,axis=0))
    
    time_EM_mean = lead_time_var[0][0:np.asarray(model_var[0]).shape[0]:3]

#    time_model_var_mean = lead_time_var[0][~np.isnan(model_var_mean).any(axis=0)]
 #   model_var_mean = np.asarray(model_var_mean)[~np.isnan(model_var_mean)]
################    

### save calendar day etc. for scatter plot
    calendar_day.append(calday)
    initialisation_day.append(ini_day)
    calendar_month.append(calmon)
    calendar_year.append(year)
    initialisation_hh.append(hh) 
########


#### calculate the difference between obs and model
    diff_var, Diff_var_0, \
        Diff_var_1, Diff_var_2, Diff_var_3, \
        Diff_var_4, Diff_var_5, Diff_var_6, \
        Diff_var_7, Diff_var_8, Diff_var_9  = calc_diff(diff_var, \
                                                       model_var, Variable_obs, Diff_var_0,\
                                                       Diff_var_1, Diff_var_2, Diff_var_3, \
                                                       Diff_var_4, Diff_var_5, Diff_var_6, \
                                                       Diff_var_7, Diff_var_8, Diff_var_9,var=var)

    if var == 'WD':
        for ens_memb in range(0,10):
            if (diff_var[ens_memb] < -180).any() == True:
                diff_var[ens_memb][np.where(diff_var[ens_memb] < -180)] = \
                diff_var[ens_memb][np.where(diff_var[ens_memb] < -180)]+360
                
            elif (diff_var[ens_memb] > 180).any() == True:
                diff_var[ens_memb][np.where(diff_var[ens_memb] > 180)] = \
                    diff_var[ens_memb][np.where(diff_var[ens_memb] > 180)]-360

#########################################
### save daily variable
    if var == 'PP' or var == 'T2' or var == 'SP':
        model_var_0.append(model_var[0][:,0])
        model_var_1.append(model_var[1][:,0])
        model_var_2.append(model_var[2][:,0])
        model_var_3.append(model_var[3][:,0])
        model_var_4.append(model_var[4][:,0])
        model_var_5.append(model_var[5][:,0])
        model_var_6.append(model_var[6][:,0])
        model_var_7.append(model_var[7][:,0])
        model_var_8.append(model_var[8][:,0])
        model_var_9.append(model_var[9][:,0])
    elif var == 'WD' or var == 'WS':
        model_var_0.append(model_var[0])#[:,0])
        model_var_1.append(model_var[1])#[:,0])
        model_var_2.append(model_var[2])#[:,0])
        model_var_3.append(model_var[3])#[:,0])
        model_var_4.append(model_var[4])#[:,0])
        model_var_5.append(model_var[5])#[:,0])
        model_var_6.append(model_var[6])#[:,0])
        model_var_7.append(model_var[7])#[:,0])
        model_var_8.append(model_var[8])#[:,0])
        model_var_9.append(model_var[9])#[:,0])
    
    obs_var.append(Variable_obs)
    
#########################################    
    if hh < 10:
        hh = '0%s' %hh
    else:
        hh = hh
#########################################
    for ens_memb in range(0,10):
        if var == 'PP' or var == 'T2' or var == 'SP':
            lead_time_var[ens_memb] = lead_time_var[ens_memb][~np.isnan(model_var[ens_memb][:,0])]
        elif var == 'WD' or var == 'WS':
            lead_time_var[ens_memb] = lead_time_var[ens_memb][~np.isnan(model_var[ens_memb])]
        model_var[ens_memb]     = model_var[ens_memb][~np.isnan(model_var[ens_memb])]
    
    
    
        fn_var[ens_memb].close()
###########

        
        
        

##### PLOTTING ############################################
    fig_name = '%s%s%s_%s.png' %(year,month,ini_day,hh)
    
    title = 'initalised: %s, %s %s %s %s UTC' %(calday,ini_day,calmon,year,hh)
    xdays = ['%s-%s-%s' %(year,month,ini_day), '',
        6,'',12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+1), '',
        6,'', 12,'', 18,'',
        '%s-%s-%s' %(year,month,ini_day+2)]
####### plot variable    
    lgd = plt_variable(lead_time_var,model_var,Variable_obs,time_EM_mean, np.asarray(model_var_mean)[0,:],var=var,
                       xdays=xdays, title = title)
    if savefig == 1:
 #       plt.savefig('%s/%s' % (figdir_var, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        plt.savefig('%s/%s' % (figdir_var, fig_name), format = form, bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_var,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

###################################################################    
#### plot scatter observation vs. model
    if day == '20':
        cc = no1
    elif day == '21':
        cc = no2
    elif day == '22':
        cc = no3
    elif day == '23':
        cc = no4
    elif day == '24':
        cc = no5
    elif day == '25':
        cc = no6
    elif day == '26':
        cc = no7
    label = '%s, %s %s %s %s UTC' %(calday,ini_day, 
                                        calmon,year,hh)
    plt_scatter_obs_model(Variable_obs, model_var,cc,var=var,label=label)  
    if savefig == 1:
        plt.savefig('%s/obs_model_%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/obs_model_%s' %(figdir_scatter,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()
     

###################################################################    
#### plot scatter difference
 
    
    plt_scatter_diff(diff_var, lead_time_var, Variable_obs, cc, var=var,label=label)
    if savefig == 1:
        plt.savefig('%s/diff_%s' % (figdir_scatter, fig_name))
        print('plot saved: %s/diff_%s' %(figdir_scatter,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/numpy/lib/nanfunctions.py:1423: RuntimeWarning: Degrees of freedom <= 0 for slice.
  keepdims=keepdims)


plot saved: ../../Figures/MEPS/wind_speed/20161220_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161220_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161220_00.png
plot saved: ../../Figures/MEPS/wind_speed/20161221_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161221_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161221_00.png
plot saved: ../../Figures/MEPS/wind_speed/20161222_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161222_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161222_00.png
plot saved: ../../Figures/MEPS/wind_speed/20161223_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161223_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161223_00.png
plot saved: ../../Figures/MEPS/wind_speed/20161224_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wi

In [13]:
###################################
model_var_all_day = dict()

model_var_all_day[0]=(model_var_0)
model_var_all_day[1]=(model_var_1)
model_var_all_day[2]=(model_var_2)
model_var_all_day[3]=(model_var_3)
model_var_all_day[4]=(model_var_4)
model_var_all_day[5]=(model_var_5)
model_var_all_day[6]=(model_var_6)
model_var_all_day[7]=(model_var_7)
model_var_all_day[8]=(model_var_8)
model_var_all_day[9]=(model_var_9)

In [14]:
model_2124 = []
obs_var_2124 = []
for ens_memb in range(0,10):
    model_2124.append(np.asarray(model_var_all_day[ens_memb])[1:4,:np.asarray(obs_var)[1:4,:].shape[0]])
    obs_var_2124.append(np.asarray(obs_var)[1:4,:np.asarray(obs_var)[1:4,:np.asarray(obs_var)[1:4,:].shape[0]].shape[0]])

In [15]:
from scipy import stats

def regression_all_EM(model_var_all_day,obs_var,day):
    model_2112 = []
    obs_var_2112 = []
    for ens_memb in range(0,10):
        model_2112.append(np.asarray(model_var_all_day[ens_memb])[day,:np.asarray(obs_var)[day,:].shape[0]])
        obs_var_2112.append(np.asarray(obs_var)[day,:np.asarray(obs_var)[day,:].shape[0]])


#    print(np.asarray(obs_var_2112).shape)
 #   print(np.asarray(model_2112).shape)
    PP_obs_flat_21 = np.ndarray.flatten(np.asarray(obs_var_2112))
    PP_model_flat_21 = np.ndarray.flatten(np.asarray(model_2112))

    x_obs = np.asarray(PP_obs_flat_21)
    y_obs = np.asarray(PP_model_flat_21)
    mask_obs = ~np.isnan(x_obs) & ~np.isnan(y_obs)
    gradient_obs, intercept_obs, r_value, p_value, std_err = stats.linregress(x_obs[mask_obs],y_obs[mask_obs])

    return(PP_obs_flat_21,PP_model_flat_21,x_obs,gradient_obs,intercept_obs, r_value, p_value, std_err)

In [16]:
PP_obs_flat = dict()
PP_model_flat = dict()
x_obs = dict()
gradient_obs = dict()
intercept_obs = dict()
r_obs = dict()
p_obs = dict()
std_err_obs = dict()


for day in range(0,len(obs_var)):
#    print(day)
    PP_obs_flat[day],PP_model_flat[day],x_obs[day],gradient_obs[day],intercept_obs[day], r_obs[day], p_obs[day], std_err_obs[day] \
        = regression_all_EM(model_var_all_day,obs_var,day)


In [17]:
###################################################################    
#### plot scatter observation vs. model
lgd = plt_scatter_all_days(obs_var,model_var_all_day,var=var, day_range=np.arange(0,len(obs_var)-1),
                               in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh,
                          x_obs=x_obs,intercept_obs=intercept_obs,gradient_obs=gradient_obs,
                          r_value=r_obs)
fig_name = 'obs_model_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
    #plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form, bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161220_26_00.png


In [18]:
### 20.12. - 23.12. 
lgd = plt_scatter_all_days(obs_var,model_var_all_day,var=var, day_range=np.arange(0,4),
                               in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh,
                          x_obs=x_obs,intercept_obs=intercept_obs,gradient_obs=gradient_obs,
                          r_value=r_obs)
fig_name = 'obs_model_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
#    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form, bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161220_23_00.png


In [19]:
### 21.12. - 23.12. 
lgd = plt_scatter_all_days(obs_var,model_var_all_day,var=var, day_range=np.arange(1,4),
                               in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh,
                          x_obs=x_obs,intercept_obs=intercept_obs,gradient_obs=gradient_obs,
                          r_value=r_obs)
fig_name = 'obs_model_%s%s%s_%s_%s.png' %(calendar_year[1],month,initialisation_day[1],initialisation_day[4-1],hh)
if savefig == 1:
#    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.savefig('%s/%s' % (figdir_scatter, fig_name),  bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161221_23_00.png


In [20]:
### 24.12. - 26.12.
lgd = plt_scatter_all_days(obs_var,model_var_all_day,var=var, day_range=np.arange(4,len(obs_var)-1),
                               in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh,
                          x_obs=x_obs,intercept_obs=intercept_obs,gradient_obs=gradient_obs,
                          r_value=r_obs)
fig_name = 'obs_model_%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
#    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form, bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/obs_model_20161224_26_00.png


In [21]:
###################################################################    
#### plot scatter difference
Diff_obs_model_var = dict()

Diff_obs_model_var[0]=(Diff_var_0)
Diff_obs_model_var[1]=(Diff_var_1)
Diff_obs_model_var[2]=(Diff_var_2)
Diff_obs_model_var[3]=(Diff_var_3)
Diff_obs_model_var[4]=(Diff_var_4)
Diff_obs_model_var[5]=(Diff_var_5)
Diff_obs_model_var[6]=(Diff_var_6)
Diff_obs_model_var[7]=(Diff_var_7)
Diff_obs_model_var[8]=(Diff_var_8)
Diff_obs_model_var[9]=(Diff_var_9)

In [22]:
pd.DataFrame(Diff_obs_model_var[0])

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,-4.968185,-2.978170,1.513670,-0.586409,-0.557546,-2.931011,-3.518873,-3.515867,-2.452683,-0.547578,...,4.113329,3.384974,2.296088,11.538753,2.315254,3.587811,5.104802,8.476078,5.028444,2.618438
1,5.694490,4.937621,5.943408,5.516130,4.435672,2.214324,6.208077,7.301652,5.959482,7.271976,...,4.761346,5.472980,5.906161,7.581699,3.243031,2.506237,7.963942,5.653478,6.685457,6.530997
2,3.131244,3.889514,2.081288,5.080657,4.332537,6.158203,4.234903,6.871894,6.345481,4.568073,...,6.561809,5.628264,2.238375,5.493147,7.160100,3.554041,4.789400,4.613247,0.812031,4.516561
3,4.837647,2.489633,6.111595,4.028598,4.551957,5.037531,5.774673,3.565132,3.311582,-1.838816,...,9.873964,10.409782,11.961993,9.879919,8.514526,8.066063,6.303224,5.149913,7.723685,3.665725
4,5.943059,2.650818,4.488514,2.232794,5.062233,4.105370,9.026114,2.082672,6.604826,6.047121,...,1.307617,0.779051,4.771045,5.696416,6.126441,1.554540,4.674362,2.201392,4.361148,-5.343395
5,6.150337,4.659823,3.918119,6.625199,9.406995,5.724767,4.199461,1.078742,2.150717,7.188227,...,7.843365,7.317858,6.927308,4.644978,9.086191,7.433490,4.680638,-0.386060,0.547056,5.815614
6,3.589239,-0.136350,6.298603,1.415783,1.343473,1.356382,4.402235,-0.583108,5.666663,8.268285,...,2.816655,-1.346394,-1.374549,-0.583314,4.164517,4.360581,0.088428,-1.038779,-1.767504,-1.288209
7,2.210757,-3.190361,0.563971,-6.327617,-4.450051,0.109474,-3.255902,-2.382230,-2.371229,-0.668189,...,-0.028511,-1.612034,0.382104,1.332899,0.043263,-0.816070,1.621219,-1.067729,-0.975360,4.080793


In [23]:
avg_perc = []

In [24]:
model_diff = []
diff_perc = []

for i in range(6,7):
    print(np.asarray(obs_var)[i,12])
    for ens_memb in range(0,10):
        print('model %s' %ens_memb,np.asarray(model_var_all_day[ens_memb])[i,12])
        model_diff.append(np.asarray(model_var_all_day[ens_memb])[i,12])
        print('Diff [perc], %s' %ens_memb, 
              (np.asarray(model_var_all_day[ens_memb])[i,12]-np.asarray(obs_var)[i,12])/np.asarray(obs_var)[i,12]*100)
              #(np.asarray(Diff_obs_model_var[ens_memb])[i,12]/np.asarray(obs_var)[i,12])*100)
        diff_perc.append((np.asarray(Diff_obs_model_var[ens_memb])[i,12]/np.asarray(obs_var)[i,12])*100)
    print('Avg [perc]', np.nanmean(diff_perc))
    avg_perc.append(np.nanmean(diff_perc))
#print(np.nanmean(avg_perc))
np.nanmean(model_diff)

11.3
model 0 18.8181
Diff [perc], 0 66.5318919494
model 1 21.4401
Diff [perc], 1 89.7352944433
model 2 18.3131
Diff [perc], 2 62.0626533981
model 3 22.5621
Diff [perc], 3 99.6642762581
model 4 20.611
Diff [perc], 4 82.3979943199
model 5 18.7265
Diff [perc], 5 65.7214375724
model 6 19.0081
Diff [perc], 6 68.2131775713
model 7 17.0366
Diff [perc], 7 50.7665414726
model 8 20.9243
Diff [perc], 8 85.1706715812
model 9 20.8201
Diff [perc], 9 84.2484600776
Avg [perc] 75.4512398644


19.825989

In [25]:
avg_perc

[75.451239864383098]

In [26]:
np.nanmean(np.asarray(avg_perc)[2:4])

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


nan

In [27]:
model_var_all_day[0]

[array([  3.23181534,   0.92182952,   3.71367002,   3.313591  ,
          3.54245377,   2.86898899,   1.38112652,   0.7841332 ,
          1.04731655,   3.35242248,   2.62047577,   2.38374996,
          5.48539162,   5.65734529,   3.81020331,   1.97543585,
          3.92267942,   3.67418814,   5.12262917,   5.14663792,
          3.63407707,   4.9174428 ,   9.81883621,   9.97523689,
         10.07511806,  10.63937092,  10.26501274,  10.38252354,
          8.58603954,   9.65589046,  11.07098103,   8.86693573,
         11.44164562,  12.86144543,  11.34171867,   9.67417908,
          6.90273714,  10.35145283,  12.74576759,  12.61332893,
          9.48497391,   8.89608765,  17.43875313,  15.1152544 ,
         14.28781128,  14.70480156,  16.87607765,  13.62844372,
         13.21843815,  11.24436951,  10.91003609,  11.8912344 ,
         12.23930264,  13.01486683,  14.27849388,  14.54627609,
         13.93540668,  14.23996735,  14.32502556,  14.55796242,
         16.21046829,  16.08426285,  17.

In [28]:
avg_perc = []

In [29]:
obs_var_10per = (np.asarray(obs_var)/90)*100

In [30]:
model_diff = []
diff_perc = []

for i in range(6,7):
    print(obs_var_10per[i,12])
    for ens_memb in range(0,10):
        print('model %s' %ens_memb,np.asarray(model_var_all_day[ens_memb])[i,12])
        print('Diff [perc], with 10',
              (np.asarray(model_var_all_day[ens_memb])[i,12] - obs_var_10per[i,12])/obs_var_10per[i,12] *100)
        model_diff.append(np.asarray(model_var_all_day[ens_memb])[i,12])
 #       print('Diff [perc], %s' %ens_memb, 
  #            (np.asarray(model_var_all_day[ens_memb])[i,12]-np.asarray(obs_var)[i,12])/np.asarray(obs_var)[i,12]*100)
   #           #(np.asarray(Diff_obs_model_var[ens_memb])[i,12]/np.asarray(obs_var)[i,12])*100)
        diff_perc.append((np.asarray(model_var_all_day[ens_memb])[i,12] - obs_var_10per[i,12])/obs_var_10per[i,12] *100)
    print('Avg [perc]', np.nanmean(diff_perc))
    avg_perc.append(np.nanmean(diff_perc))
#print(np.nanmean(avg_perc))
np.nanmean(model_diff)

12.5555555556
model 0 18.8181
Diff [perc], with 10 49.8787027545
model 1 21.4401
Diff [perc], with 10 70.761764999
model 2 18.3131
Diff [perc], with 10 45.8563880583
model 3 22.5621
Diff [perc], with 10 79.6978486323
model 4 20.611
Diff [perc], with 10 64.158194888
model 5 18.7265
Diff [perc], with 10 49.1492938151
model 6 19.0081
Diff [perc], with 10 51.3918598141
model 7 17.0366
Diff [perc], with 10 35.6898873253
model 8 20.9243
Diff [perc], with 10 66.6536044231
model 9 20.8201
Diff [perc], with 10 65.8236140698
Avg [perc] 57.9061158779


19.825989

In [31]:
avg_perc

[57.906115877944806]

In [32]:
np.nanmean(np.asarray(avg_perc)[2:4])

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: Mean of empty slice
  """Entry point for launching an IPython kernel.


nan

In [33]:
pd.DataFrame(np.asarray(avg_perc)[1:7])

,0


In [34]:
lgd = plt_all_day_diff(Diff_obs_model_var,lead_time_var,Variable_obs,var,np.arange(0,len(Diff_obs_model_var[0])-1),
                      in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'diff_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()


### 20.12. - 23.12. 
lgd = plt_all_day_diff(Diff_obs_model_var,lead_time_var,Variable_obs,var,np.arange(0,4),
                      in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'diff_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

### 21.12. - 23.12. 
lgd = plt_all_day_diff(Diff_obs_model_var,lead_time_var,Variable_obs,var,np.arange(1,4),
                      in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'diff_%s%s%s_%s_%s.png' %(calendar_year[1],month,initialisation_day[1],initialisation_day[4-1],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

### 24.12. - 26.12.
lgd = plt_all_day_diff(Diff_obs_model_var,lead_time_var,Variable_obs,var,np.arange(4,len(Diff_obs_model_var[0])-1),
                      in_day=initialisation_day, cal_mon=calendar_month,
                              cal_year=calendar_year,in_hh=initialisation_hh)
fig_name = 'diff_%s%s%s_%s_%s.png' %(calendar_year[4],month,initialisation_day[4],initialisation_day[len(obs_var)-2],hh)
if savefig == 1:
    plt.savefig('%s/%s' % (figdir_scatter, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    print('plot saved: %s/%s' %(figdir_scatter,fig_name))
elif savefig == 0:
    plt.show()
plt.close()

plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161220_26_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161220_23_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161221_23_00.png
plot saved: ../../Figures/MEPS/scatter_MEPS_obs/wind_speed/diff_20161224_26_00.png


In [35]:
def mean_abs_error(Diff_obs_model_var,ens_memb,start_hour,end_hour):
    MAE_0 = []
    ME_0 = []
    for day in range(0,len(Diff_obs_model_var[ens_memb])):
        ME_0.append(np.nanmean((np.asarray(Diff_obs_model_var[ens_memb])[day,start_hour:end_hour])))
        MAE_0.append( np.nanmean(np.abs(np.asarray(Diff_obs_model_var[ens_memb])[day,start_hour:end_hour])))
    return(MAE_0, ME_0)
MAE = dict()
ME = dict()
MAE_12 = dict()
ME_12 = dict()
MAE_12_24 = dict()
ME_12_24 = dict()

ME6_25 = dict()
MAE0_24 = dict() 
ME0_24 = dict()
MAE0_30 = dict() 
ME0_30 = dict()
MAE0_36 = dict() 
ME0_36 = dict()
MAE0_42 = dict() 
ME0_42 = dict()
for ens_memb in range(0,10):
    MAE[ens_memb], ME6_25[ens_memb] = mean_abs_error(Diff_obs_model_var,ens_memb,
                                                 6,25)
    MAE0_24[ens_memb], ME0_24[ens_memb] = mean_abs_error(Diff_obs_model_var,ens_memb,
                                                 0,25)
    MAE0_30[ens_memb], ME0_30[ens_memb] = mean_abs_error(Diff_obs_model_var,ens_memb,
                                                 0,31)
    MAE0_36[ens_memb], ME0_36[ens_memb] = mean_abs_error(Diff_obs_model_var,ens_memb,
                                                 0,37)
    MAE0_42[ens_memb], ME0_42[ens_memb] = mean_abs_error(Diff_obs_model_var,ens_memb,
                                                 0,43)
    
    ME[ens_memb] = np.mean([ME0_24[ens_memb], ME0_30[ens_memb], ME0_36[ens_memb], ME0_42[ens_memb]],axis = 0)
    
    if var == 'PP':
        MAE_12[ens_memb], ME_12[ens_memb] = mean_abs_error(Diff_obs_model_var,ens_memb,
                                                 6,(6+13))
        MAE_12_24[ens_memb], ME_12_24[ens_memb] = mean_abs_error(Diff_obs_model_var,ens_memb,
                                                 6,25)

/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:5: RuntimeWarning: Mean of empty slice
  """
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in absolute
  
/Volumes/SANDISK128/Applications/anaconda/envs/py36ECMWF/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: Mean of empty slice
  


In [36]:
def plt_mean_abs_error(MAE,var):
    
    ### plot mean absolute error
    fig = plt.figure(figsize=(20,5))
    ax = fig.add_subplot(111)

    ax.plot(MAE[0][1:7],'k', linewidth=4, label = 'deterministic')
    ax.plot(MAE[1][1:7],color = spagh.memb_col, linestyle='-', label = 'ensemble member')
    for ens_memb in range(2,10):
        ax.plot(MAE[ens_memb][1:7],color = spagh.memb_col, linestyle='-', label = '_nolegend_')

    ### fine tuning
    lgd = ax.legend(loc='upper center', #bbox_to_anchor=(0.5, -0.37),
              fancybox=True, shadow=True, ncol=3, fontsize=spagh.label_fs)
    frame = lgd.get_frame()
    frame.set_facecolor('white')

    # xaxis
    ax.set_xlim(-0.25,5.25)
    ax.set_xlabel('Dec 2016', fontsize=spagh.label_fs)
    ax.set_xticks(np.arange(0,6))
    ax.set_xticklabels([ 21, 22, 23, 24, 25, 26], fontsize=spagh.tick_fs)

    ax.axhline(0, color=spagh.vert_col, linewidth=3)
    #ax.axvline(1, color=spagh.vert_col, linewidth=3)
    #ax.axvline(2, color=spagh.vert_col, linewidth=3)
    #ax.axvline(3, color=spagh.vert_col, linewidth=3)
    #ax.axvline(4, color=spagh.vert_col, linewidth=3)
    #ax.axvline(5, color=spagh.vert_col, linewidth=3)
    #ax.axvline(6, color=spagh.vert_col, linewidth=3)



    if var == 'PP':
        # yaxis
        ax.set_ylim(-0.25, 25)
        ax.set_yticks(np.arange(0,27.5,2.5))
        ax.set_yticklabels([0, '',  5,'',10,'',15,'',20,'',25], fontsize=spagh.tick_fs)
        ax.set_title('12$\,$h Precipiation acc.', fontsize=spagh.fontsize)
        ax.set_ylabel('MAE [mm]', fontsize=spagh.label_fs)
    elif var ==  'SP':
        # yaxis
        ax.set_ylim(-0.25, 4.5)
        ax.set_yticks(np.arange(0,4.5,.25))
        ax.set_yticklabels([0, '', 0.5 ,'',1.0,'',1.5,'',2.0,'',2.5,'',3,'',3.5,'',4], fontsize=spagh.tick_fs)
        ax.set_title('Sea Level Pressure', fontsize=spagh.fontsize)
        ax.set_ylabel('MAE [hPa]', fontsize=spagh.label_fs)
    elif var == 'T2':
        # yaxis
        ax.set_ylim(-0.25, 2.5)
        ax.set_yticks(np.arange(0,2.75,.25))
        ax.set_yticklabels([0, '', 0.5 ,'',1.0,'',1.5,'',2.0,'',2.5,'',3,''], fontsize=spagh.tick_fs)
        ax.set_title('2$\,$m Air Temperature', fontsize=spagh.fontsize)
        ax.set_ylabel('MAE [K]', fontsize=spagh.label_fs)
    elif var == 'WD':
        # yaxis 
        ax.set_ylim(-0.25,45)
        ax.set_yticks(np.arange(0,50,5))
        ax.set_yticklabels([0, '',10,'',20,'',30,'',40,'',50], fontsize=spagh.tick_fs)
        ax.set_title('10$\,$m Wind direction', fontsize=spagh.fontsize)
        ax.set_ylabel('MAE [$^\circ$]', fontsize=spagh.label_fs)
    elif var == 'WS':
        # yaxis
        ax.set_ylim(-.25,12.5)
        ax.set_yticks(np.arange(0,12.5,1))
        ax.set_yticklabels([0, '',2,'',4,'',6,'',8,'',10,'',12,''], fontsize=spagh.tick_fs)
        ax.set_title('10$\,$m Wind speed', fontsize=spagh.fontsize) 
        ax.set_ylabel('MAE [m$\,$s$^{-1}$]', fontsize=spagh.label_fs)

    plt.tight_layout()

In [37]:
if var != 'PP':
    plt_mean_abs_error(MAE,var)
    fig_name = 'MAE_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_error, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_error,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

plot saved: ../../Figures/MEPS/error/wind_speed/MAE_20161220_26_00.png


In [38]:
if var == 'PP':
    plt_mean_abs_error(MAE_12,var)
    fig_name = 'MAE_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_error, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_error,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

#    plt_mean_abs_error(MAE_12_24,var)
 #   fig_name = 'MAE12_24_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
  #  if savefig == 1:
   #     plt.savefig('%s/%s' % (figdir_error, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    #    print('plot saved: %s/%s' %(figdir_error,fig_name))
#    elif savefig == 0:
 #       plt.show()
  #  plt.close()

In [39]:
for day in range(0,1):
    for ens_memb in range(0,10):
        print('2%s Dec' %day,'MAE: %s' %ens_memb, MAE[ens_memb][day])

20 Dec MAE: 0 2.49370684875
20 Dec MAE: 1 3.4526453884
20 Dec MAE: 2 2.6874261243
20 Dec MAE: 3 4.51489309754
20 Dec MAE: 4 2.00301260267
20 Dec MAE: 5 2.08012643031
20 Dec MAE: 6 3.83639478683
20 Dec MAE: 7 3.2441944497
20 Dec MAE: 8 2.73890141078
20 Dec MAE: 9 2.24980391775


In [40]:
def plt_mean_error(ME,var):
    ### plot mean error
    fig = plt.figure(figsize=(20,5))
    ax = fig.add_subplot(111)

    ax.plot(ME[0][1:7],'k', linewidth=4, label = 'deterministic')
    ax.plot(ME[1][1:7],color = spagh.memb_col, linestyle='-', label = 'ensemble member')
    for ens_memb in range(2,10):
        ax.plot(ME[ens_memb][1:7],color = spagh.memb_col, linestyle='-', label = '_nolegend_')

    ### fine tuning
    lgd = ax.legend(loc='upper center', #bbox_to_anchor=(0.5, -0.37),
              fancybox=True, shadow=True, ncol=3, fontsize=spagh.label_fs)
    frame = lgd.get_frame()
    frame.set_facecolor('white')

    # xaxis
    ax.set_xlim(-0.25,5.25)
    ax.set_xlabel('Dec 2016', fontsize=spagh.label_fs)
    ax.set_xticks(np.arange(0,6))
    ax.set_xticklabels([21, 22, 23, 24, 25, 26], fontsize=spagh.tick_fs)

    ax.axhline(0, color=spagh.vert_col, linewidth=3)



    if var == 'PP':
        # yaxis
        ax.set_ylim(0, 25)
        ax.set_yticks(np.arange(-5,27.5,2.5))
        ax.set_yticklabels([-5, '',0, '',  5,'',10,'',15,'',20,'',25], fontsize=spagh.tick_fs)
        ax.set_title('12$\,$h Precipitation acc. (00+24,+30,+36,+42$\,$UTC)', fontsize=spagh.fontsize)
        ax.set_ylabel('ME [mm]', fontsize=spagh.label_fs)
    elif var ==  'SP':
        # yaxis
        ax.set_ylim(-0.25, 3)
        ax.set_yticks(np.arange(0,3.5,.25))
        ax.set_yticklabels([0, '', 0.5 ,'',1.0,'',1.5,'',2.0,'',2.5,'',3,''], fontsize=spagh.tick_fs)
        ax.set_title('Sea Level Pressure (00+24,+30,+36,+42$\,$UTC)', fontsize=spagh.fontsize)
        ax.set_ylabel('ME [hPa]', fontsize=spagh.label_fs)
    elif var == 'T2':
        # yaxis
        ax.set_ylim(-2., 1)
        ax.set_yticks(np.arange(-2,1.75,.25))
        ax.set_yticklabels([-2,'',-1.5,'',-1,'',-0.5 ,'',0, '', 0.5 ,'',1.0,'',1.5,'',2.0,'',2.5,'',3,''], fontsize=spagh.tick_fs)
        ax.set_title('2$\,$m Air Temperature (00+24,+30,+36,+42$\,$UTC)', fontsize=spagh.fontsize)
        ax.set_ylabel('ME [K]', fontsize=spagh.label_fs)
    elif var == 'WD':
        # yaxis 
        ax.set_ylim(-17.5,20)
        ax.set_yticks(np.arange(-20,22.5,2.5))
        ax.set_yticklabels([-20,'',-15,'',-10,'',-5 ,'',0, '', 5 ,'',10,'',15,'',20,'',25,'',30,''], fontsize=spagh.tick_fs)
        ax.set_title('10$\,$m Wind direction (00+24,+30,+36,+42$\,$UTC)', fontsize=spagh.fontsize)
        ax.set_ylabel('ME [$^\circ$]', fontsize=spagh.label_fs)
    elif var == 'WS':
        # yaxis
        ax.set_ylim(-.25,12.5)
        ax.set_yticks(np.arange(0,12.5,1))
        ax.set_yticklabels([0, '',2,'',4,'',6,'',8,'',10,'',12,''], fontsize=spagh.tick_fs)
        ax.set_title('10$\,$m Wind speed (00+24,+30,+36,+42$\,$UTC)', fontsize=spagh.fontsize) 
        ax.set_ylabel('ME [m$\,$s$^{-1}$]', fontsize=spagh.label_fs)

    plt.tight_layout()

In [41]:
if var != 'PP':
    plt_mean_error(ME,var)
    fig_name = 'ME_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_error, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_error,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

plot saved: ../../Figures/MEPS/error/wind_speed/ME_20161220_26_00.png


In [42]:
if var == 'PP':
    plt_mean_error(ME_12,var)
    fig_name = 'ME_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
    if savefig == 1:
        plt.savefig('%s/%s' % (figdir_error, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
        print('plot saved: %s/%s' %(figdir_error,fig_name))
    elif savefig == 0:
        plt.show()
    plt.close()

#    plt_mean_error(ME_12_24,var)
 #   fig_name = 'ME12_24_%s%s%s_%s_%s.png' %(calendar_year[0],month,initialisation_day[0],initialisation_day[len(obs_var)-2],hh)
  #  if savefig == 1:
   #     plt.savefig('%s/%s' % (figdir_error, fig_name), format = form,bbox_extra_artists=(lgd,), bbox_inches='tight')
    #    print('plot saved: %s/%s' %(figdir_error,fig_name))
#    elif savefig == 0:
 #       plt.show()
  #  plt.close()

In [43]:
for ens_memb in range(0,10):
    print('%s' %ens_memb, MAE_12[ens_memb][5])

KeyError: 0